# Ideas

- load states.h5
- Search nearest kNN with annoy


In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import annoy
import h5py

from annoy import AnnoyIndex

## How to build index etc (don't re-run!)

In [2]:
# Load the data
f = h5py.File("S2S/states.h5", "r")
cstar = f["decoder_out"]#f["cstar"]
print(cstar.shape)

(20010, 55, 500)


In [12]:
cstar = f["encoder_out"]

In [5]:
small_copy.close()

In [6]:
# Delete cstar to create a small copy
small_copy = h5py.File("S2S/states_small.h5", "w")
small_copy.create_group('src')
f.copy('src', small_copy['src'])
small_copy.create_group('tgt')
f.copy('tgt', small_copy['tgt'])
small_copy.create_group('attn')
f.copy('attn', small_copy['attn'])
small_copy.close()

In [13]:
# Create the index
t = AnnoyIndex(cstar.shape[2], metric='angular')

In [14]:
# Add samples to index, takes long!
for samplenum, sample in enumerate(cstar):
    for tokennum, tokencontext in enumerate(sample):
        index = cstar.shape[1] * samplenum + tokennum
        t.add_item(index, tokencontext)
        
        if (cstar.shape[1] * samplenum + tokennum) % 1000 == 0:
            print("value #", cstar.shape[1] * samplenum + tokennum)
#     if samplenum > 200:
#         break

value # 0
value # 1000
value # 2000
value # 3000
value # 4000
value # 5000
value # 6000
value # 7000
value # 8000
value # 9000
value # 10000
value # 11000
value # 12000
value # 13000
value # 14000
value # 15000
value # 16000
value # 17000
value # 18000
value # 19000
value # 20000
value # 21000
value # 22000
value # 23000
value # 24000
value # 25000
value # 26000
value # 27000
value # 28000
value # 29000
value # 30000
value # 31000
value # 32000
value # 33000
value # 34000
value # 35000
value # 36000
value # 37000
value # 38000
value # 39000
value # 40000
value # 41000
value # 42000
value # 43000
value # 44000
value # 45000
value # 46000
value # 47000
value # 48000
value # 49000
value # 50000
value # 51000
value # 52000
value # 53000
value # 54000
value # 55000
value # 56000
value # 57000
value # 58000
value # 59000
value # 60000
value # 61000
value # 62000
value # 63000
value # 64000
value # 65000
value # 66000
value # 67000
value # 68000
value # 69000
value # 70000
value # 71000
value

value # 554000
value # 555000
value # 556000
value # 557000
value # 558000
value # 559000
value # 560000
value # 561000
value # 562000
value # 563000
value # 564000
value # 565000
value # 566000
value # 567000
value # 568000
value # 569000
value # 570000
value # 571000
value # 572000
value # 573000
value # 574000
value # 575000
value # 576000
value # 577000
value # 578000
value # 579000
value # 580000
value # 581000
value # 582000
value # 583000
value # 584000
value # 585000
value # 586000
value # 587000
value # 588000
value # 589000
value # 590000
value # 591000
value # 592000
value # 593000
value # 594000
value # 595000
value # 596000
value # 597000
value # 598000
value # 599000
value # 600000
value # 601000
value # 602000
value # 603000
value # 604000
value # 605000
value # 606000
value # 607000
value # 608000
value # 609000
value # 610000
value # 611000
value # 612000
value # 613000
value # 614000
value # 615000
value # 616000
value # 617000
value # 618000
value # 619000
value # 62

value # 1094000
value # 1095000
value # 1096000
value # 1097000
value # 1098000
value # 1099000
value # 1100000


In [15]:
# Build trees
t.build(10)

True

In [16]:
# Save index to file
t.save("encoder.ann")

True

## How to use the search

In [12]:
# Load the index from file
u = AnnoyIndex(500)
u.load("states.ann")

True

In [13]:
# Load the file with states etc
f = h5py.File("S2S/states_small.h5", "r")

In [14]:
# Define loader for dictionary and load them
def load_dict(fname):
    ix2w = {}
    with open(fname, "r") as f:
        for l in f:
            cline = l.split()
            ix2w[int(cline[0])] = cline[1]
    ix2w[0] = "<unk>"
    return ix2w

In [15]:
src_dict = load_dict("S2S/src.dict")
tgt_dict = load_dict("S2S/tgt.dict")

In [17]:
# Test functionality by giving index as input
def get_closest(ix, k=10, ignore_same_tgt=False):
    if ignore_same_tgt:
        interval_min = ix // 55 * 55
        return [k for k in u.get_nns_by_item(ix,k+55, search_k = 100000) 
                if not interval_min <= k <= interval_min+55][:k]
    else:
        return u.get_nns_by_item(ix,k, search_k = 100000)
print(get_closest(1,5))
print(get_closest(1,5, True))

[2, 398476, 126556, 398478, 1127119]
[398476, 126556, 398478, 1127119, 813452]


In [18]:
# Index is stretched out, need to find src/tgt index
def convert_result_to_correct_index(oldix):
    return oldix // 55, oldix % 55
convert_result_to_correct_index(101)

(1, 46)

In [19]:
# Transform tokens, ignore padding (1)
def ix2text(array, vocab, highlight=-1):
    tokens = []
    for ix, t in enumerate(array):
        if ix == highlight:
            tokens.append("___" + vocab[t] + "___")
        elif t != 1:
            tokens.append(vocab[t])
    return " ".join(tokens)


In [20]:
# Compute length of a sentence when ignoring padding
def compute_sent_length(array):
    return np.sum([1 for t in array if t != 1])

In [21]:
# Convert a result for an index
def convert_result(ix):
    sentIx, tokIx = convert_result_to_correct_index(ix)
    # Get raw list of tokens
    src_in = f['src']['src'][sentIx]
    tgt_in = f['tgt']['tgt'][sentIx]
    # Convert to text
    src = ix2text(src_in, src_dict)
    tgt = ix2text(tgt_in, tgt_dict, tokIx)
    attn = f['attn']['attn'][sentIx]
    src_len = compute_sent_length(src_in)
    tgt_len = compute_sent_length(tgt_in)
    attn = attn[:tgt_len,:src_len]
    print(src)
    print(tgt)
    return src, tgt, attn
src, tgt, attn = convert_result(123)

&quot; Two soldiers came up to me and told me that if I refuse to sleep with them , they will kill me . They beat me and ripped my clothes .
<s> Also kam ich nach Südafrika " , erzählte eine Frau namens Grace ___dem___ Human Rights <unk> Gerry Simpson , der die Probleme der <unk> Flüchtlinge in Südafrika untersucht .


### Case study

We have word number 500, and want to know which it was closest to

In [26]:
investigated_num = 11

In [27]:
curr_res = get_closest(investigated_num,5,True)
print(curr_res)

[845440, 1232403, 119646, 1138024, 536939]


In [28]:
# First we print what it was
_ = convert_result(investigated_num)

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
<s> Es geht nicht an , dass über <unk> , deren ___Inhalt___ , Zweck und Ausmaß vorher nicht bestimmt ist , zusammen mit den nationalen Bürokratien das <unk> des Europäischen Parlaments ausgehebelt wird .


In [29]:
# Then, the closest one's
for r in curr_res:
    _ = convert_result(r)
    print()

European political parties can make a key contribution to the development of a political and democratic Europe and can help the European Institutions meet the challenges that European issues and , above all , the future geography of Europe , are going to pose .
<s> Die europäischen politischen Parteien können in entscheidender Weise an der Ausarbeitung eines politischen und demokratischen Entwurfs in Europa mitarbeiten und die europäischen Institutionen dabei unterstützen , den Herausforderungen zu begegnen , die durch europäische ___Themen___ und vor allem durch die geographische Dimension des künftigen Europa aufgeworfen werden .

In order to achieve this , we must encourage private initiative , we must help local SMEs , which often suffer as a result of their isolation and communication difficulties and we must maintain services .
<s> Voraussetzung hierfür ist , dass die Privatinitiative gefördert und den lokalen KMU , die häufig unter der ___Abgeschiedenheit___ und den <unk> leiden